# weather data

In [29]:
import pandas as pd
import os

train_size = 1008 # 1000
test_size = 250 # 250
horizon = 50 # 50

path = os.path.join('results',f'weather_datatrain_{train_size}_test_{test_size}_horizon_{horizon}_weather__WITH_FINETUNING_results.csv')


df = pd.read_csv(path)
# get name of the file without the .csv
name_of_file = os.path.basename(path).split('.')[0]
name_of_file = name_of_file + "_weather"
name_of_file

'weather_datatrain_1008_test_250_horizon_50_weather__WITH_FINETUNING_results_weather'

In [30]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Initialize lists to store metrics
maes = []
rmses = []
mapes = []

# Group by 'RollingOrigin' and calculate metrics for each group
for _, group in df.groupby('RollingOrigin'):
    y_true = group['Actual']
    y_pred = group['TimeGPT']
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    
    # Append metrics to lists
    maes.append(mae)
    rmses.append(rmse)
    mapes.append(mape)

# Calculate the mean and standard deviation of the metrics
mae_mean = np.mean(maes)
mae_std = np.std(maes)
rmse_mean = np.mean(rmses)
rmse_std = np.std(rmses)
mape_mean = np.mean(mapes)
mape_std = np.std(mapes)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Source': [name_of_file],
    'MAE': [mae_mean],
    'MAE_std': [mae_std],
    'RMSE': [rmse_mean],
    'RMSE_std': [rmse_std],
    'MAPE': [mape_mean],
    'MAPE_std': [mape_std]
})
print(results)

# save the results
results.to_csv(os.path.join('error_metrics',f'{name_of_file}.csv'), index=False)

                                              Source       MAE   MAE_std  \
0  weather_datatrain_1008_test_250_horizon_50_wea...  2.342171  0.988625   

       RMSE  RMSE_std          MAPE      MAPE_std  
0  2.808237  1.060507  1.819037e+14  2.444959e+14  


# Climate data

In [4]:
import pandas as pd
import os

train_size = 1000
test_size = 50
horizon = 10
model1 = 'timegpt-1'
model2 = 'timegpt-1-long-horizon'

path = os.path.join('results','climate_data', f'train_{train_size}_test_{test_size}_horizon_{horizon}_modeltype_{model1}_climate_results.csv')

df = pd.read_csv(path)
# get name of the file without the .csv
name_of_file = os.path.basename(path).split('.')[0]
name_of_file
df

,date,TimeGPT,Actual,RollingOrigin
0,1953-06-01,0.575735,0.343228,0
1,1953-07-01,0.706922,0.357698,0
2,1953-08-01,0.802228,0.185581,0
3,1953-09-01,0.636237,-0.048259,0
4,1953-10-01,0.464142,0.060318,0
...,...,...,...,...
405,1957-02-01,0.137916,-0.137413,40
406,1957-03-01,0.137567,0.060404,40
407,1957-04-01,0.142735,0.091391,40
408,1957-05-01,0.143780,0.002730,40


In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Initialize lists to store metrics
maes = []
rmses = []
mapes = []

def calculate_error_metrics_with_mase(df, name_of_file):
    maes, rmses, mapes, mases = [], [], [], []
    
    # Group by 'RollingOrigin' and calculate metrics for each group
    for _, group in df.groupby('RollingOrigin'):
        y_true = group['Actual'].values
        y_pred = group['TimeGPT'].values

        # Calculate metrics
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mape = mean_absolute_percentage_error(y_true, y_pred)
        mase = np.mean([abs(y_true[i] - y_pred[i]) / (abs(y_true[i] - y_true[i - 1]) / (len(y_true) - 1)) for i in range(1, len(y_true))])

        # Append metrics to lists
        maes.append(mae)
        rmses.append(rmse)
        mapes.append(mape)
        mases.append(mase)

    # Calculate the mean and standard deviation of the metrics
    mae_mean = np.mean(maes)
    mae_std = np.std(maes)
    rmse_mean = np.mean(rmses)
    rmse_std = np.std(rmses)
    mape_mean = np.mean(mapes)
    mape_std = np.std(mapes)
    mase_mean = np.mean(mases)
    mase_std = np.std(mases)

    # Create a DataFrame to store the results
    results = pd.DataFrame({
        'Source': [name_of_file],
        'MAE': [mae_mean],
        'MAE_std': [mae_std],
        'RMSE': [rmse_mean],
        'RMSE_std': [rmse_std],
        'MAPE': [mape_mean],
        'MAPE_std': [mape_std],
        'MASE': [mase_mean],
        'MASE_std': [mase_std]
    })
    print(results)
    return results

results = calculate_error_metrics_with_mase(df, name_of_file)

# save the results
results.to_csv(os.path.join('error_metrics',f'{name_of_file+"TESTTTT"}.csv'), index=False)

                                              Source       MAE   MAE_std  \
0  train_1000_test_50_horizon_10_modeltype_timegp...  0.153126  0.064758   

       RMSE  RMSE_std     MAPE  MAPE_std       MASE   MASE_std  
0  0.181398  0.065473  2.12848  1.937849  67.569216  64.243391  


# all files

In [11]:
# load all results files for weather data
import os
import pandas as pd

# Initialize an empty list to store the results
results = []

# Iterate over all files in the 'results' directory
for file in os.listdir('results/weather_data'):
    # Check if the file is a CSV file
    if file.endswith('.csv'):
        # Read the file into a DataFrame
        df = pd.read_csv(os.path.join('results/weather_data', file))
        # Append the DataFrame to the list
        results.append(df)

results[1]

,DateTime,TimeGPT,Actual,RollingOrigin
0,2024-04-11 16:00:00,12.883865,13.7,0
1,2024-04-11 17:00:00,12.562263,13.5,0
2,2024-04-11 18:00:00,11.709729,12.8,0
3,2024-04-11 19:00:00,11.166174,11.2,0
4,2024-04-11 20:00:00,10.894749,10.0,0
...,...,...,...,...
10045,2024-04-21 21:00:00,4.143261,2.5,200
10046,2024-04-21 22:00:00,3.790940,1.4,200
10047,2024-04-21 23:00:00,3.484714,0.9,200
10048,2024-04-22 00:00:00,3.457513,0.8,200


In [19]:
results[1]["TimeGPT"].values

array([12.88386536, 12.56226254, 11.70972919, ...,  3.48471403,
        3.45751286,  3.39269066])